# Example: Metabolite workflow

This notebook provides insights on how to use the tool if you have list of metabolites.
Our current data sources include:
* MolMeDB

In [1]:
# Import modules
import pandas as pd

from pyBiodatafuse import id_mapper
from pyBiodatafuse.annotators import molmedb, opentargets
from pyBiodatafuse.constants import MOLMEDB_COMPOUND_PROTEIN_COL, OPENTARGETS_DISEASE_COL
from pyBiodatafuse.utils import combine_sources

# Load list of metabolites/chemicals/compounds

In [2]:
metabolites_of_interest = """100208
10040286
10041551
10025195
5291"""
# metabolites_of_interest = """CHEMBL1201583
# CHEMBL941
# """
# metabolites_of_interest = "5291"
# metabolites_of_interest = "10025195"

metabolite_list = metabolites_of_interest.split("\n")
len(metabolite_list)

5

In [3]:
data_input = pd.DataFrame(metabolite_list, columns=["identifier"])
data_input.head()

,identifier
0,100208
1,10040286
2,10041551
3,10025195
4,5291


### Entity resolution using BridgeDB

In [4]:
bridgdb_df, bridgdb_metadata = id_mapper.bridgedb_xref(
    identifiers=data_input,
    input_species="Human",
    input_datasource="PubChem Compound",  # ChEMBL compound
    output_datasource="All",
)
bridgdb_df.head()

,identifier,identifier.source,target,target.source
0,100208,PubChem-compound,90560,ChemSpider
1,100208,PubChem-compound,100208,PubChem Compound
2,100208,PubChem-compound,HMDB0244377,HMDB
3,100208,PubChem-compound,OFDNQWIFNXBECV-UHFFFAOYSA-N,InChIKey
4,100208,PubChem-compound,C11280,KEGG Compound


### Transporter inhibited from MolMeDB

In [5]:
#test = molmedb.get_compound_gene_inhibitor(bridgedb_df=bridgdb_df)
#test.head()

In [6]:
(
    molmedb_transporter_inhibited_df,
    molmedb_transporter_inhibited_metadata,
) = molmedb.get_compound_gene_inhibitor(bridgedb_df=bridgdb_df)
molmedb_transporter_inhibited_df.head()

,identifier,identifier.source,target,target.source,MolMeDB_transporter_inhibited
0,100208,PubChem-compound,OFDNQWIFNXBECV-UHFFFAOYSA-N,InChIKey,"[{'uniprot_trembl_id': 'P08183', 'hgnc_symbol'..."
1,10025195,PubChem-compound,LEJRLSZVESQKJK-UHFFFAOYSA-N,InChIKey,"[{'uniprot_trembl_id': nan, 'hgnc_symbol': nan..."
2,10040286,PubChem-compound,FYGREZKTJIXWIH-UHFFFAOYSA-N,InChIKey,"[{'uniprot_trembl_id': nan, 'hgnc_symbol': nan..."
3,10041551,PubChem-compound,OVVBIIBBRZVPAL-UHFFFAOYSA-N,InChIKey,"[{'uniprot_trembl_id': 'P23975', 'hgnc_symbol'..."
4,5291,PubChem-compound,KTUFNOKKBVMGRW-UHFFFAOYSA-N,InChIKey,"[{'uniprot_trembl_id': 'O15244', 'hgnc_symbol'..."


In [7]:
molmedb_transporter_inhibited_df[MOLMEDB_COMPOUND_PROTEIN_COL][4]

[{'uniprot_trembl_id': 'O15244',
  'hgnc_symbol': 'SLC22A2',
  'source_pmid': '23241029'},
 {'uniprot_trembl_id': 'Q9UNQ0',
  'hgnc_symbol': 'ABCG2',
  'source_pmid': '15155841, 18678495, 19932960'}]